In [1]:
# pip install imgaug


In [1]:
import cv2
import imgaug.augmenters as iaa
import os
import xml.etree.ElementTree as ET
import json
from torchvision import transforms 
from PIL import Image
import numpy as np

from dacl10k import utils
import matplotlib.pyplot as plt
from PIL import Image
import cv2

c:\aryan\Cracks\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Sep img and ann

def apply_transforms(image_path, annotation_path, save_image_dir, save_annotation_dir):
    # Load image
    image = Image.open(image_path).convert("RGB")

    # Load LabelMe annotation
    with open(annotation_path, "r") as annotation_file:
        annotation_data = json.load(annotation_file)

    # Extract polygon coordinates
    polygons = annotation_data["shapes"]

    # Define PyTorch transforms
    horizontal_flip = transforms.RandomHorizontalFlip(p=1.0)
    vertical_flip = transforms.RandomVerticalFlip(p=1.0)

    # Apply horizontal flip to image and annotations
    flipped_image_h = horizontal_flip(image)

    # Update annotation data for horizontal flip
    for i, segment in enumerate(polygons):
        points = np.array(segment["points"]).reshape(-1, 2)
        flipped_points_h = np.array([[image.width - point[0], point[1]] for point in points]).reshape(-1, 2).tolist()
        polygons[i]["points"] = flipped_points_h

    # Save augmented images
    save_image_path_h = os.path.join(save_image_dir, "flipped_horizontal_" + os.path.basename(image_path))
    flipped_image_h.save(save_image_path_h)

    # Save augmented annotations
    save_annotation_path_h = os.path.join(save_annotation_dir, "flipped_horizontal_" + os.path.basename(annotation_path))
    with open(save_annotation_path_h, "w") as annotation_file_h:
        json.dump(annotation_data, annotation_file_h)

    # Apply vertical flip to image and annotations
    flipped_image_v = vertical_flip(horizontal_flip(image))

    # Update annotation data for vertical flip
    for i, segment in enumerate(polygons):
        points = np.array(segment["points"]).reshape(-1, 2)
        flipped_points_v = np.array([[point[0], image.height - point[1]] for point in points]).reshape(-1, 2).tolist()
        polygons[i]["points"] = flipped_points_v

    # Save augmented images
    save_image_path_v = os.path.join(save_image_dir, "flipped_vertical_" + os.path.basename(image_path))
    flipped_image_v.save(save_image_path_v)

    # Save augmented annotations
    save_annotation_path_v = os.path.join(save_annotation_dir, "flipped_vertical_" + os.path.basename(annotation_path))
    with open(save_annotation_path_v, "w") as annotation_file_v:
        json.dump(annotation_data, annotation_file_v)

def read_images_and_annotations(image_folder, annotation_folder, output_image_folder, output_annotation_folder):
    # Iterate over all files in the image folder
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            # Construct paths for image and annotation
            image_path = os.path.join(image_folder, filename)
            annotation_path = os.path.join(annotation_folder, filename.replace(".jpg", ".json").replace(".jpeg", ".json"))

            # Check if the corresponding annotation file exists
            if os.path.exists(annotation_path):
                # Apply transforms and save augmented images and annotations
                apply_transforms(image_path, annotation_path, output_image_folder, output_annotation_folder)

# Validation Augmentation

In [4]:
if __name__ == "__main__":
    
    # Specify paths
    image_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\640x640\images\validation"
    annotation_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\640x640\annotations\validation"
    output_image_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\640x640\images_aug\validation"
    output_annotation_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\640x640\annotations_json_aug\validation"

    # Iterate over all images and annotations
    read_images_and_annotations(image_folder, annotation_folder, output_image_folder, output_annotation_folder)

## Validation Checks

### Orginal Image

In [8]:
image_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images\validation\dacl10k_v2_validation_0005.jpg"
annotation_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json\validation\dacl10k_v2_validation_0005.json"

image = cv2.imread(image_file)

with open(annotation_file, "rb") as f:
    data = json.load(f)
    
    polygon_points = np.array(data["shapes"][0]["points"], dtype=np.int32)

# print(polygon_points)

# Draw the polygon on the image
cv2.polylines(image, [polygon_points], isClosed=True, color=(255,0,0), thickness=2) # Display the image
cv2.imwrite("Segformer_checks/Polygon Points Validation OG.png", image)

True

### Horizontal Image

In [9]:
image_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images_aug\validation\flipped_horizontal_dacl10k_v2_validation_0005.jpg"
annotation_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json_aug\validation\flipped_horizontal_dacl10k_v2_validation_0005.json"

image = cv2.imread(image_file)

with open(annotation_file, "rb") as f:
    data = json.load(f)
    
    polygon_points = np.array(data["shapes"][0]["points"], dtype=np.int32)

# print(polygon_points)

# Draw the polygon on the image
cv2.polylines(image, [polygon_points], isClosed=True, color=(255,0,0), thickness=2) # Display the image
cv2.imwrite("Segformer_checks/Polygon Points Validation Horizontal.png", image)

True

### Vertical Image

In [10]:
image_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images_aug\validation\flipped_vertical_dacl10k_v2_validation_0005.jpg"
annotation_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json_aug\validation\flipped_vertical_dacl10k_v2_validation_0005.json"

image = cv2.imread(image_file)

with open(annotation_file, "rb") as f:
    data = json.load(f)
    
    polygon_points = np.array(data["shapes"][0]["points"], dtype=np.int32)

# print(polygon_points)

# Draw the polygon on the image
cv2.polylines(image, [polygon_points], isClosed=True, color=(255,0,0), thickness=2) # Display the image
cv2.imwrite("Segformer_checks/Polygon Points Validation Vertical.png", image)

True

# Train Augmentation

In [11]:
if __name__ == "__main__":
    
    # Specify paths
    image_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images\train"
    annotation_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json\train"
    output_image_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images_aug\train"
    output_annotation_folder = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json_aug\train"

    # Iterate over all images and annotations
    read_images_and_annotations(image_folder, annotation_folder, output_image_folder, output_annotation_folder)

## Train Check

### Original Image

In [12]:
image_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\images\train\dacl10k_v2_train_0000.jpg"
annotation_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\annotations_labelme\train\dacl10k_v2_train_0000.json"

image = cv2.imread(image_file)

with open(annotation_file, "rb") as f:
    data = json.load(f)
    
    polygon_points = np.array(data["shapes"][0]["points"], dtype=np.int32)

# print(polygon_points)

# Draw the polygon on the image
cv2.polylines(image, [polygon_points], isClosed=True, color=(255,0,0), thickness=2) # Display the image
cv2.imwrite("Segformer_checks/Polygon Points Train OG.png", image)

True

### Horizontal

In [15]:
image_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images_aug\train\flipped_horizontal_dacl10k_v2_train_0000.jpg"
annotation_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json_aug\train\flipped_horizontal_dacl10k_v2_train_0000.json"

image = cv2.imread(image_file)

with open(annotation_file, "rb") as f:
    data = json.load(f)
    
    polygon_points = np.array(data["shapes"][0]["points"], dtype=np.int32)

# print(polygon_points)

# Draw the polygon on the image
cv2.polylines(image, [polygon_points], isClosed=True, color=(255,0,0), thickness=2) # Display the image
cv2.imwrite("Segformer_checks/Polygon Points Train Horizontal.png", image)

True

### Vertical

In [16]:
image_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\images_aug\train\flipped_vertical_dacl10k_v2_train_0000.jpg"
annotation_file = r"C:\aryan\Cracks\data\dacl10k_v2_devphase\resized\data\annotations_json_aug\train\flipped_vertical_dacl10k_v2_train_0000.json"

image = cv2.imread(image_file)

with open(annotation_file, "rb") as f:
    data = json.load(f)
    
    polygon_points = np.array(data["shapes"][0]["points"], dtype=np.int32)

# print(polygon_points)

# Draw the polygon on the image
cv2.polylines(image, [polygon_points], isClosed=True, color=(255,0,0), thickness=2) # Display the image
cv2.imwrite("Segformer_checks/Polygon Points Train Vertical.png", image)

True